## Assignment 1 (50 marks)
#### =====================================================================================================
### Deadline: 09/14 11:59 pm
#### =====================================================================================================

## Decision Tree Classification

### Problem 1 (25 marks)

`lab01_dataset_1.csv` contains a mixture of numerical and categorical data. Your task will be to write a function `my_ID3()` (without using the sklearn decision tree library) which can create a decision tree for the given dataset using the ID3 algorithm.

### 1.a (5 marks)

ID3 cannot handle continuous numerical data. Perform the necessary operations to convert the continuous-valued attribute to boolean attributes (refer to page 40 of Decision Tree.pdf). Do not use any of the sklearn library for this task. Display the updated dataset after handling continuous-valued attribute.

In [8]:
import pandas as pd
df = pd.read_csv("lab01_dataset_1.csv")
threshold = df["Score"].median() 
df["Score_high"] = df["Score"] > threshold
print("Threshold used for Score:", threshold)
print(df)


Threshold used for Score: 69.0
       Mood  Effort  Score Output  Score_high
0     Happy     Low     35    Yes       False
1     Happy  Medium     91     No        True
2     Happy    High     52     No       False
3   Neutral     Low     83     No        True
4   Neutral  Medium     48     No       False
5   Neutral    High     61     No       False
6       Sad     Low     40    Yes       False
7       Sad  Medium     98     No        True
8       Sad    High     73    Yes        True
9     Happy     Low     44    Yes       False
10    Happy  Medium     86     No        True
11    Happy    High     39    Yes       False
12    Happy     Low     66     No       False
13  Neutral  Medium     75    Yes        True
14  Neutral    High     50     No       False
15      Sad     Low     69     No       False
16      Sad  Medium     70    Yes        True
17      Sad    High     95     No        True
18      Sad     Low     80    Yes        True


### 1.b (6 marks)

Next, you will have to ensure that the newly obtained dataset is optimal and free of errors. Take appropriate actions based on the outcomes of the following steps:

Check if the dataset has any missing values. (2 marks)<br>
Check if the dataset has any redundant or repeated input sample. (2 marks)<br>
Check if the dataset has any contradicting <input, output> pairs. (2 marks)

Do note that you need to perform all the above steps programmatically and not simply by visual examination.

In [10]:
import pandas as pd
# (i) Missing values
missing_counts = df.isnull().sum()
has_missing = missing_counts.sum() > 0

# (ii) Redundant / repeated input samples
# Consider inputs = all columns except target (last column)
target_col = df.columns[-1]
input_cols = [c for c in df.columns if c != target_col]

duplicates = df.duplicated(subset=input_cols, keep=False)
redundant_samples = df[duplicates].sort_values(by=input_cols)

# (iii) Contradicting <input, output> pairs
# Find rows with same inputs but different outputs
grouped = df.groupby(input_cols)[target_col].nunique()
contradictions = grouped[grouped > 1]

# Gather results
results = {
    "threshold_used": threshold,
    "has_missing": has_missing,
    "missing_counts": missing_counts.to_dict(),
    "num_redundant_samples": int(duplicates.sum()),
    "redundant_examples": redundant_samples.head(10).to_dict(orient="records"),
    "num_contradictions": int(contradictions.shape[0]),
    "contradictions": contradictions.to_dict()
}
print(results)

{'threshold_used': np.float64(69.0), 'has_missing': np.False_, 'missing_counts': {'Mood': 0, 'Effort': 0, 'Score': 0, 'Output': 0, 'Score_high': 0}, 'num_redundant_samples': 0, 'redundant_examples': [], 'num_contradictions': 0, 'contradictions': {}}


### 1.c (10 marks)

Your function `my_ID3()` should operate in a manner such that after ever round of decision making, it will output the attributes and its associated gain, with a message stating “Attribute X with Gain = Y is chosen as the decision attribute”. You can obviously create other functions as needed for completing `my_ID3()`.

In [ ]:
def my_ID3(dataset): # the input to the function is the final dataset you obtain after 1.b.
    # iterate until decision tree is created.
        # ID3 algorithm involves two mathematial computation.
        # Entropy and Information Gain (check pages 12 to 15 of Decision Tree.pdf).
        # Attributes with maximum Information Gain is chosen as decision nodes to split the data.
        # Students should print out each decision nodes alongwith their respective Information Gain.
    return decision_tree

### 1.d (4 marks)

Display the decision tree. The representation of the decision tree is upto you. You can choose either a textual representation or a graphical one; either is fine.

### Problem 2 (25 marks)

`lab01_dataset_2.csv` has a mixture of numerical and categorical data. For this problem, you will use sklearn's [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for the classification task.

### 2.a (5 marks)

Encode all the categorial data into numerical data using `One-hot Encoding` and display the new dataset.

### 2.b (10 marks)

Using the encoded dataset, perform the supervised learning using [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Employ the [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) of 80-20 during the training.

### 2.c (5 marks)

After the training is complete, show the results by predicting the class of the test set. Display the results of the prediction and test set side-by-side.

### 2.d (5 marks)

Display the decision tree; it can be either a textual representation or a graphical representation.